# Finding similar documents

This notebook aims to use a pipeline to find similar documents.

The pipeline is as follows:

graph LR

    A[Documents] --> B[Tokenization]
    B --> C[Shingling]
    C --> D[MinHashing]
    D --> E[LSH]
    E --> F[Similarity Check]

In [40]:
import os
import itertools 
import numpy as np
from tqdm import tqdm

## Data collection
First step that we need to do is to find data to check for similarity.
We found a dataset with 10 different types of documents with 100 docs each.
This was downloaded and put into the current working directory in a folder called `data`

```python
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jensenbaxter/10dataset-text-document-classification")

print("Path to dataset files:", path)

```


After downloading the data, the documents have to be read from memory everytime we want to process them

```python
docs = read_documents("data")
```

In [41]:
def read_documents(path):
    documents = []
    for root, _, files in os.walk(path):
        for file in files:
            if file.endswith('.txt'):
                with open(os.path.join(root, file), 'r') as f:
                    documents.append(f.read())
    return documents
    

## Tokenization

This step involves converting the docs into tokens. 

E.g. "A cat eats" -> [0,1,2]

Where 
vocab = {
    0: a, 
    1: cat,
    2: eats,
}

We can use a more complicated tokenizer, e.g. BPE. This would speed up processing larger quantities of data since a number would represent a larger chunk. 

Nevertheless,
using a word level tokenizer is fine for the purpose of this exercise. Our main goal is to convert the documents into integer representations, allowing for efficient and more general computations. E.g. by only using numbers we can restrict memory usage with uint16 etc which allows for quicker computes. 

In this tokenizer, we are compressing the vocab slightly more by ignoring capitalizations. 


In [42]:
class Tokenizer:
    def __init__(self):
        self.vocabulary = {}
    
    def fit(self, documents):
        for doc in documents:
            doc = doc.lower()
            for word in doc.split():
                if word not in self.vocabulary:
                    self.vocabulary[word] = len(self.vocabulary)
        return self

    def encode(self, text):
        return [self.vocabulary[word.lower()] for word in text.split()]

    def decode(self, tokens):
        return ' '.join([self.vocabulary[token] for token in tokens])

## Shingling

shingle size 2

abcba -> [ab, bc, cb, ab]

unique set -> {ab, bc, cb}


TODO:
To further optimize the shingling process, we should use less overlap.

shingle size 3, with 1 overlap

abcdefghij -> [abc, cde, efg, ghi, ij]



In [43]:

class Shingling:
    @staticmethod
    def create_shingles(text, k):
        # Create k-shingles as tuples
        shingles = {tuple(text[i:i+k]) for i in range(len(text) - k + 1)}
        
        # Hash each shingle and store in sorted order
        hashed_shingles = {hash(shingle) for shingle in shingles}
        
        return hashed_shingles


In [44]:
Shingling.create_shingles([1,2,3,4,5], 3)



{-3165226637586315787, 529344067295497451, 4003026094496801395}

## MinHashing

This step can be done in a lot of different ways, the main idea is to compress doc shingle representations.



Below two different methods are shown. 
1. using permutations to simulate hashfunctions
    The permutations are used as masks for the original representation. Then, from the masked representation we take the min value, and use that as the compressed information for that particular permutation. This is done for n steps, to create a compressed n vector representation of the single
2. using hashfunctions 
    generating n different hashfunctions
        generating random params to universal hashing formula: 
            fixed seed. Its not necessary since we're keeping the same for the entire sig matrix generation.
        
            `((ax + b) % p) % m`

            x: is the integer representation of each input (this case shingles)

            a: randomly generated. needs to be in the range [1, p-1]. Can't be zero since then the function would be simplified to `(b % p) % m`, hashing some inputs to the same values, which is undesirable. Should be less than p, since otherwise some inputs could get hashed to the same values. 

            b: randomly generated.
            same as 'a' but can be zero since the input is not removed. Would be simplified to (ax % p) % m
            
            m: the universal size
            It would be fine to not use `% m`. Doing so will ensure the output is in the desired range M instead of P.

            p: prime number
            the next prime bigger than the universal size (number of unique shingles), to bin each shingle to its own. It ensures good distribution properties. 

        
        




In [45]:
from typing import List

class MinHashing:
    def __init__(self):
        self.unique_shingles = set()
        
    def fit(self, sets:List[set]):
        for set_ in sets:
            self.unique_shingles.update(set_)

    # def characteristic_vector(self, shingles: set):
    #     shingles_array = np.array(list(self.unique_shingles))
    #     return np.isin(shingles_array, list(shingles)).astype(np.int8)
    
    # def signature(self, n, shingles:set):
    #     np.random.seed(n)
        
    #     permutations = np.array([np.random.permutation(len(self.unique_shingles)) for _ in range(n)])
    #     characteristic_vector = self.characteristic_vector(shingles)
    #     permuted_shingles = permutations * characteristic_vector
    #     minhash = np.array([np.min(row[row != 0]) if np.any(row != 0) else 0 for row in permuted_shingles])
    #     return minhash

    def _hash_function(self, x: int, a: int, b: int, p: int, m: int) -> int:
        return ((a * x + b) % p) % m
        
    def signature_matrix(self, n: int, shingles: List[set]):
        self.fit(shingles)
        m = len(self.unique_shingles)
        p = next_prime(m)
        
        np.random.seed(42)
        
        # generate n different hashfunctions
        hash_params = [(np.random.randint(1, p), np.random.randint(0, p)) for _ in range(n)]
        
        signature_matrix = np.full((len(shingles), n), np.inf)
        
        # universal index
        shingle_to_int = {shingle: idx for idx, shingle in enumerate(self.unique_shingles)}
        
        # create signature for each document
        for i, doc_shingles in enumerate(tqdm(shingles, desc="MinHashing")):
            
            for shingle in doc_shingles:
                x = shingle_to_int[shingle]
                
                # apply each hashfunction to get the signature
                for j, (a, b) in enumerate(hash_params):
                    hash_value = self._hash_function(x, a, b, p, m)
                    signature_matrix[i, j] = min(signature_matrix[i, j], hash_value)
        
        return signature_matrix.astype(np.int32)

def next_prime(n):
    def is_prime(num):
        if num < 2:
            return False
        for i in range(2, int(num ** 0.5) + 1):
            if num % i == 0:
                return False
        return True
    
    next_num = n + 1
    while not is_prime(next_num):
        next_num += 1
    return next_num


    

## LSH

Locality-sensitive hashing basically mean that we just check a small portion of the signatures and if that portion is the same for two signatures, we consider them to be similar documents. The larger bands we have, more stuff needs to be the same for it to be a match, making us pickier. 

We hash every band for each signature

If there's a signature band collision, two signatures being hashed to the same bucket, we consider them being similar documents

To generate all candidate pairs we give all the combinations of collisions in the buckets


In [46]:
class LSH:
    @staticmethod
    def candidate_pairs(signatures:np.array, b, r):
        
        assert signatures.shape[1] % b == 0, "The number of columns in the signature matrix must be divisible by the number of bands"
        unique_pairs = set()
        
        band_buckets = [{} for _ in range(b)]
        
        
        for i in tqdm(range(b), desc="Processing bands"):
            band = signatures[:, i*r:(i+1)*r]
            
            # Create buckets using band hash
            band_bucket = band_buckets[i]
            for doc_idx, band_signature in enumerate(band):
                # Convert band signature to bytes for hashing
                band_hash = hash(band_signature.tobytes())
                if band_hash not in band_bucket:
                    band_bucket[band_hash] = []
                band_bucket[band_hash].append(doc_idx)
            
            # Only compare documents that hash to the same bucket
            for bucket in band_bucket.values():
                if len(bucket) > 1:  # Only process buckets with collisions
                    for j, k in itertools.combinations(bucket, 2):
                        unique_pairs.add((j, k))
        
        return unique_pairs 


    

## Similiarity Check

The similarity check used here is the jaccard similarity. The intersection of two sets divided by the union.
Depending on the representations of the sets, this will be calculated differently.

In [47]:
class CompareSets:
    @staticmethod
    def jaccard(a:set, b:set):
        return len(a.intersection(b)) / len(a.union(b))

In [48]:
class CompareSignatures:
    @staticmethod
    def jaccard(a:np.array, b:np.array):
        return np.mean(a == b)


## Putting it all together

Below we are using all the previously explained methods to find similar documents

The parameters N, B, R can be chosen for a more or less granular search

K tells how many tokens are used per shingle. A higher K would mean that it represents a more unique part of the document, and vice versa

In [75]:
import time

# Parameters
N = 50  # Number of hashfunctions
B = 10  # Number of bands
R = 5   # Number of rows per band
K = 7   # Number of tokens per shingle

# Read documents
print("Starting similarity search...")
start = time.time()
docs = read_documents("data")
read_time = time.time() - start
print(f"✓ Reading documents: {read_time:.2f}s")

# Tokenization
start = time.time()
tokenizer = Tokenizer().fit(docs)
tokenization_time = time.time() - start
print(f"✓ Tokenization: {tokenization_time:.2f}s")

# Shingling
start = time.time()
shingled_docs = [Shingling.create_shingles(tokenizer.encode(doc), K) for doc in tqdm(docs, desc="Shingling")]
shingling_time = time.time() - start
print(f"✓ Shingling: {shingling_time:.2f}s")

# MinHashing
start = time.time()
sig_matrix = MinHashing().signature_matrix(N, shingled_docs)
minhashing_time = time.time() - start
print(f"✓ MinHashing: {minhashing_time:.2f}s")

# LSH
start = time.time()
candidate_pairs = list(LSH.candidate_pairs(sig_matrix, b=B, r=R))
lsh_time = time.time() - start
print(f"✓ LSH: {lsh_time:.2f}s")


# Results
total_time = read_time + tokenization_time + shingling_time + minhashing_time + lsh_time
print(f"\nTotal search time: {total_time:.2f}s")

print("\nSimilarity Results:")
print(f"Number of candidate pairs: {len(candidate_pairs)}")

Starting similarity search...
✓ Reading documents: 0.14s
✓ Tokenization: 0.05s


Shingling: 100%|██████████| 1000/1000 [00:00<00:00, 4927.90it/s]


✓ Shingling: 0.21s


MinHashing: 100%|██████████| 1000/1000 [00:06<00:00, 151.59it/s]


✓ MinHashing: 6.67s


Processing bands: 100%|██████████| 10/10 [00:00<00:00, 2573.35it/s]

✓ LSH: 0.01s

Total search time: 7.07s

Similarity Results:
Number of candidate pairs: 20


## Check results
Below we will check our found candidate pairs, the documents and their respective similarity 

In [76]:
def view_documents(docs, x, y, max_width=70):
    def wrap_text(text, width):
        # Split text into chunks of max_width characters
        return [text[i:i+width] for i in range(0, len(text), width)]
    
    doc1_lines = docs[x].split("\n")
    doc2_lines = docs[y].split("\n")
    
    # Wrap long lines
    doc1_wrapped = [line for text in doc1_lines for line in wrap_text(text, max_width)]
    doc2_wrapped = [line for text in doc2_lines for line in wrap_text(text, max_width)]
    
    print(f"\n{'='*100}\nComparing documents {x} and {y}\n{'='*100}")
    print(f"{'Document ' + str(x):<{max_width+5}} | {'Document ' + str(y)}")
    print(f"{'-'*(max_width+5)}-+-{'-'*max_width}")
    
    for line1, line2 in itertools.zip_longest(doc1_wrapped, doc2_wrapped, fillvalue=""):
        print(f"{line1:<{max_width+5}} | {line2}")
    print()




Lets check out the candidate pairs!

In [74]:
T = 0.8 # Threshold for similarity

sorted_candidate_pairs = sorted(candidate_pairs, key=lambda x: CompareSets.jaccard(shingled_docs[x[0]], shingled_docs[x[1]]), reverse=True)
for x, y in sorted_candidate_pairs:
    if CompareSets.jaccard(shingled_docs[x], shingled_docs[y]) >= T:
        print(f"Shingle Similarity: {CompareSets.jaccard(shingled_docs[x], shingled_docs[y]):.4f}")
        print(f"Signature Similarity: {CompareSignatures.jaccard(sig_matrix[x], sig_matrix[y]):.4f}")
        view_documents(docs, x, y, max_width=50)


Shingle Similarity: 1.0000
Signature Similarity: 1.0000

Comparing documents 40 and 65
Document 40                                             | Document 65
--------------------------------------------------------+---------------------------------------------------
Boothroyd calls for Lords speaker                       | Boothroyd calls for Lords speaker
Betty Boothroyd has said the House of Lords needs       | Betty Boothroyd has said the House of Lords needs 
its own Speaker and that peers should lead the way      | its own Speaker and that peers should lead the way
 on reforming the upper chamber.                        |  on reforming the upper chamber.
Baroness Boothroyd, who was the first woman to be       | Baroness Boothroyd, who was the first woman to be 
Commons Speaker, said she believed Tony Blair init      | Commons Speaker, said she believed Tony Blair init
iated reforms without a clear outcome in mind. "No      | iated reforms without a clear outcome in mind. "No
w we h